In [1]:
from hyper import *
from alignment import *
from datasets.preprecossing import *
from core import *
from datasets.loading import *
from datasets.hc_dataset import *
from datasets.balance_dataset import *
from utils.linkage import *
from model.balancehc import balancehc

from utils.poincare import *
import scib
import shutil

In [2]:
import os
import numpy as np
import pandas as pd
import scib

import scanpy as sc
from pathlib import Path
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from os.path import exists
def get_count_data(adata,counts_location=None):
    
    data = adata.layers[counts_location].copy() if counts_location else adata.X.copy()
    if not isinstance(data, np.ndarray):
        data= data.toarray()
    data_df = pd.DataFrame(data,index=adata.obs_names,columns=adata.var_names).transpose()
    return data_df


def check_paths(output_folder,output_prefix=None):
    # Create relative path
    output_path = os.path.join(os.getcwd(), output_folder)

    # Make sure that the folder exists
    Path(output_path).mkdir(parents=True, exist_ok=True)

    if os.path.exists(os.path.join(output_path, f"{output_prefix}assigned_locations.csv")):
        print("\033[91mWARNING\033[0m: Running this will overwrite previous results, choose a new"
              " 'output_folder' or 'output_prefix'")
    return output_path
def remove_batch_effect(pseudo_bulk, bulk_adata, out_dir, project='',batch_effect=True):
    """
    Remove batch effect between pseudo_bulk and input bulk data.

    Parameters
    ----------
    pseudo_bulk : anndata.AnnData
        An :class:`~anndata.AnnData` containing the pseudo expression.
    bulk_adata : anndata.AnnData
        An :class:`~anndata.AnnData` containing the input expression.
    out_dir : string, optional
        The path to save the output file.
    project : string, optional
        The prefix of output file.
        
    Returns
    -------
    Returns the expression after removing batch effect.

    """
    out_dir = check_paths(out_dir+'/batch_effect')
    if batch_effect:
        if exists(f'{out_dir}/{project}_batch_effected.txt'):
            print(f'{out_dir}/{project}_batch_effected.txt already exists, skipping batch effect.')
            bulk_data = pd.read_csv(f"{out_dir}/{project}_batch_effected.txt",sep='\t').T
        else:
            
            save=True
            # check path, file will be stored in out_dir+'/batch_effect'
            pseudo_bulk_df = get_count_data(pseudo_bulk)
            input_bulk_df= get_count_data(bulk_adata)

            bulk = pd.concat([pseudo_bulk_df,input_bulk_df], axis=1)

            cells = np.append(pseudo_bulk.obs_names, bulk_adata.obs_names)
            batch = np.append(np.ones((1,len(pseudo_bulk.obs_names))), np.ones((1,len(bulk_adata.obs_names)))+1)
            if save:
                bulk.to_csv(out_dir+f"/{project}_before_batch_effected.txt",sep='\t')
            meta = pd.DataFrame({"batch": batch,"cells":cells})
            # get r script path
            robjects.r.source('./combat.R')
            pandas2ri.activate()
            robjects.r.run_combat(bulk, meta, out_dir, project)
            # stop auto trans from pandas to r dataframe
            pandas2ri.deactivate()
            # add layer
            if exists(f'{out_dir}/{project}_batch_effected.txt'):
                bulk_data = pd.read_csv(f"{out_dir}/{project}_batch_effected.txt",sep='\t').T
                print(bulk_data.shape)
            else:
                raise ValueError('The batch_effected data is not available')
        bulk_data.clip(lower=0,inplace=True)
        # print(pseudo_bulk)
        # print(pseudo_bulk.obs_names)
        pseudo_bulk.layers["batch_effected"] = bulk_data.loc[pseudo_bulk.obs_names,:].values
        bulk_adata.layers["batch_effected"] = bulk_data.loc[bulk_adata.obs_names,:].values
    else:
        pseudo_bulk_df = get_count_data(pseudo_bulk)
        input_bulk_df= get_count_data(bulk_adata)
        bulk = pd.concat([pseudo_bulk_df,input_bulk_df], axis=1)
        bulk.to_csv(out_dir+f"/{project}_batch_effected.txt",sep='\t')

    return pseudo_bulk,bulk_adata

In [3]:
cell_path1 = './datas/23/2/BRCA_GSE110686.h5ad'
folder_path1 = './datas/testing/1/'
radius1 = 0
cell_path2 = "./datas/23/3/CRC_GSE146771.h5ad" 
folder_path2 = './datas/testing/2/'
radius2 = 0
contin = False
resolution=1
method='average'
alignment=1
n_pca=10
meta_col = 'Celltype..major.lineage.'
mst = False
epoches2 = 3
epoches1 =10
c1=0
c2=0

In [4]:
a='01'

In [5]:
cell_path1 = '../eval/datas/l/CRC_GSE146771.h5ad'
cell_path2 = '../eval/datas/l/CHOL_GSE142784.h5ad'


In [6]:
loss1 = merge_by_radius(cell_path1,folder_path1,radius1,method,meta_col)
loss1

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/orig.ident' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/nCount_RNA' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/nFeature_RNA' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/UMAP_1' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/UMAP_2' was written without encoding metadata.
  return 

1.0

In [7]:
loss2 = merge_by_radius(cell_path2,folder_path2,radius2,method,meta_col)
loss2

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/orig.ident' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/nCount_RNA' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/nFeature_RNA' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/UMAP_1' was written without encoding metadata.
  return read_elem(dataset)
/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/anndata/_io/specs/methods.py:590: OldFormatWarning: Element '/obs/UMAP_2' was written without encoding metadata.
  return 

1.0

In [8]:
adata1 = sc.read_h5ad(folder_path1+ 'adata.h5ad')
adata2 = sc.read_h5ad(folder_path2+ 'adata.h5ad')

In [9]:


preprocessing_cluster(adata1,N_pcs=n_pca,resolution=resolution)
preprocessing_cluster(adata2,N_pcs=n_pca,resolution=resolution)

inter_gene = sort_data(adata1,adata2)

tmp1 = calculate_cluster_centroid_for_genes(adata1,inter_gene,folder_path1)
tmp2 = calculate_cluster_centroid_for_genes(adata2,inter_gene,folder_path2)

# ari = adjusted_rand_score(adata1.obs['celltype'].tolist(), adata1.obs['leiden'].tolist())
# print("ARI score for adata1: ", ari)
# log1.write("ARI score for adata1: "+ str(ari)+'\n')

# ari = adjusted_rand_score(adata2.obs['celltype'].tolist(), adata2.obs['leiden'].tolist())
# print("ARI score for adata2: ", ari)
# log2.write("ARI score for adata2: "+ str(ari)+'\n')

meta_list1 = calculate_cluster_celltype(adata1);
meta_list2 = calculate_cluster_celltype(adata2);
calculate_meta_ori(folder_path1,adata1);
calculate_meta_ori(folder_path2,adata2);

2024-04-01 16:39:40.422216: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
embeddings1,nodes1 = get_Hyper_tree(folder_path1 +'datas.data',1,tmp1.shape[1]+1,0,epoches1,epoches2,save_path=folder_path1)
nos1 = build_hyper_tree_from_folder(folder_path1)


./datas/testing/1/datas.data length:16
Generating all pairs superset
	 Epoch 0 | average train loss: 0.948671
Optimization finished.
	 Epoch 1 | average train loss: 0.934838
Optimization finished.
	 Epoch 2 | average train loss: 0.933562
Optimization finished.
	 Epoch 3 | average train loss: 0.933247
Optimization finished.
	 Epoch 4 | average train loss: 0.926780
Optimization finished.
	 Epoch 5 | average train loss: 0.918468
Optimization finished.
	 Epoch 6 | average train loss: 0.917662
Optimization finished.
	 Epoch 7 | average train loss: 0.917303
Optimization finished.
	 Epoch 8 | average train loss: 0.917419
Optimization finished.
	 Epoch 9 | average train loss: 0.916684
Optimization finished.


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/utils/mst.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ij = np.empty((n - 1, 2), dtype=np.int)
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/utils/mst.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current us

	 Epoch 0 | average train loss1: 0.466618
	 Epoch 0 | average train loss2: 0.442840
	 Epoch 1 | average train loss1: 0.477281


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  origin_distance = hyp_dist(torch.tensor(self.original[i]),torch.tensor(self.original[j]));
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  origin_distance = hyp_dist(torch.tensor(self.original[int(nodef.son[0])]),torch.tensor(self.original[k])) + hyp_dist(torch.tensor(self.original[int(nodef.son[1])]),torch.tensor(self.original[k]));
/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:92: UserWarning: To copy construct from a tensor, it is recommended 

	 Epoch 1 | average train loss2: 0.442729
	 Epoch 2 | average train loss1: 0.472414
	 Epoch 2 | average train loss2: 0.440571


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/alignment.py:804: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  father_name = father_name.astype(np.int)


In [11]:
merge_points_with_c(embeddings1,nodes1,folder_path1 +'datas.data',1,tmp1.shape[1]+1,0,folder_path1,epoches2,c1,c2)
nos1 = build_hyper_tree_from_folder(folder_path1,True)
add_meta(nos1[0],meta_list1,[])
show_tree(nos1[0],color=['#184e77','#1a759f','#168aad',"#34a0a4",'#52b69a','#99d98c','#76c893','#99d98c']).show_fig()
remove_meta(nos1[0]);

/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/alignment.py:799: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  father_name = father_name.astype(np.int)


True
{'B': '#b808e7', 'CD8Tex': '#1261e5', 'Malignant': '#15e5d5', 'NK': '#639701', 'Plasma': '#f4ea02', 'CD8T': '#0f0892', 'empty': '#fff', 'Treg': '#9eb351', 'CD4Tconv': '#c01829', 'Fibroblasts': '#eeb3dc', 'Mast': '#d2682c', 'Mono/Macro': '#a3e7aa'}


In [12]:
embeddings2,nodes2 = get_Hyper_tree(folder_path2 +'datas.data',1,tmp2.shape[1]+1,0,epoches1,epoches2,save_path=folder_path2)
nos2 = build_hyper_tree_from_folder(folder_path2)


./datas/testing/2/datas.data length:22
Generating all pairs superset
	 Epoch 0 | average train loss: 0.928577
Optimization finished.
	 Epoch 1 | average train loss: 0.906281
Optimization finished.
	 Epoch 2 | average train loss: 0.904692
Optimization finished.
	 Epoch 3 | average train loss: 0.904675
Optimization finished.
	 Epoch 4 | average train loss: 0.903639
Optimization finished.
	 Epoch 5 | average train loss: 0.905249
Optimization finished.
	 Epoch 6 | average train loss: 0.904139
Optimization finished.
	 Epoch 7 | average train loss: 0.904516
Optimization finished.
	 Epoch 8 | average train loss: 0.904160
Optimization finished.
	 Epoch 9 | average train loss: 0.903831
Optimization finished.


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/utils/mst.py:4: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/utils/mst.py:10: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for addi

	 Epoch 0 | average train loss1: 0.484856
	 Epoch 0 | average train loss2: 0.445113


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:46: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



	 Epoch 1 | average train loss1: 0.503168


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:71: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/model/balancehc.py:92: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



	 Epoch 1 | average train loss2: 0.444082
	 Epoch 2 | average train loss1: 0.497186
	 Epoch 2 | average train loss2: 0.444064


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/alignment.py:804: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [13]:
merge_points_with_c(embeddings2,nodes2,folder_path2 +'datas.data',1,tmp2.shape[1]+1,0,folder_path2,epoches2,c1,c2)
nos2 = build_hyper_tree_from_folder(folder_path2,True)

True


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/alignment.py:799: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [14]:
nodes1 = build_hyper_tree_from_folder(folder_path1,True)
nodes2 = build_hyper_tree_from_folder(folder_path2,True)

nodes_merge1 = [];
nodes_merge2 = [];
add_meta(nodes1[0],meta_list1,[])
add_meta(nodes2[0],meta_list2,[])

rate,anslist,ans = run_alignment(nodes1,nodes2,folder_path1,folder_path2,meta_list1,meta_list2);


/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/alignment.py:799: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/Users/dongjiajie/Desktop/alignment/fyp/hyperalignment/final/alignment.py:799: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for a

The mininum cost for alignment is 37.21212121212122
The alignment edges list is [('30', '42'), ('29', '40'), ('26', '30'), ('22', '29'), ('21', '23'), ('11_B', '6_Endothelial'), ('6_Mono/Macro', '16_Endothelial'), ('9_Plasma', '1_Myofibroblasts'), ('8_Mono/Macro', '0_Endothelial'), ('25', '39'), ('18', '38'), ('15_Fibroblasts', '19_Hepatocytes'), ('13_Malignant', '12_Fibroblasts'), ('16', '35'), ('14_Mast', '18_CD8T'), ('4_Malignant', '3_Fibroblasts'), ('28', '41'), ('27', '36'), ('24', '32'), ('20', '27'), ('2_NK', '25'), ('0_NK', '2_Myofibroblasts'), ('5_CD8T', '9_Myofibroblasts'), ('10_NK', '4_Endothelial'), ('23', '37'), ('19', '33'), ('17', '26'), ('7_Treg', '11_Cholangiocyte'), ('1_CD4Tconv', '13_Malignant'), ('12_CD8Tex', '14_Endothelial'), ('3_CD4Tconv', '10_Hepatocytes')]
average cost for one node:0.5028665028665029

correct alignment rate:0.0


In [15]:
anslist_dist = dict(ans)
anslist_dist.keys()
def search_lineage(now,path,anss):
    path.append(now.name)
    if(now.son==[]):
        anss.append(path);
        return
    
    for i in now.son:
        search_lineage(i,path.copy(),anss);
temp1 = []
search_lineage(nodes1[0],[],temp1)
temp1
route1 = []
route2 = []

for i in temp1:
    r1 = []
    r2 = []
    for j in i:
        if j in anslist_dist.keys():
            r1.append(j)
            r2.append(anslist_dist[j])
    route1.append(r1)
    route2.append(r2)
adata1.obs.index = [i+'_1' for i in adata1.obs.index]
adata2.obs.index = [i+'_2' for i in adata2.obs.index]

In [16]:
score = [];


for i,j in zip(route1,route2):
    try:
        shutil.rmtree('./batch_effect/', ignore_errors=True)

        cells1 = [ ]
        cells2 = [ ]
        for k,t in zip(i,j):
            num1 = int(k.split('_')[0])
            num2 = int(t.split('_')[0])
            if(len(k.split('_'))>1):
                cells1.append(str(num1));
            if(len(t.split('_'))>1):
                cells2.append(str(num2));
        if(cells1==[] or cells2 ==[]):
            continue;
        
        sub_adata1 = adata1[adata1.obs['leiden'].isin(cells1),inter_gene].copy();
        sub_adata2 = adata2[adata2.obs['leiden'].isin(cells2),inter_gene].copy();

        adata1_after,adata2_after = remove_batch_effect(sub_adata1.copy(),sub_adata2.copy(),'./')
        
        sc.pp.neighbors(sub_adata1,use_rep='X')
        sc.tl.diffmap(sub_adata1)
        sub_adata1.uns['iroot'] = 0
        sc.tl.dpt(sub_adata1)
        
        
        sc.pp.neighbors(sub_adata2,use_rep='X')
        sc.tl.diffmap(sub_adata2)
        sub_adata2.uns['iroot'] = 0
        sc.tl.dpt(sub_adata2)
        
        
        adata1_after.obsm['batch_effected'] = adata1_after.layers['batch_effected']
        adata2_after.obsm['batch_effected'] = adata2_after.layers['batch_effected']
        
        sc.pp.neighbors(adata1_after,use_rep='batch_effected')
        sc.tl.diffmap(adata1_after)
        adata1_after.uns['iroot'] = 0
        sc.tl.dpt(adata1_after)

        sc.pp.neighbors(adata2_after,use_rep='batch_effected')
        sc.tl.diffmap(adata2_after)
        adata2_after.uns['iroot'] = 0
        sc.tl.dpt(adata2_after)
        
        score.append( scib.me.trajectory_conservation(sub_adata1, adata1_after, label_key="celltype"))
        score.append( scib.me.trajectory_conservation(sub_adata2, adata2_after, label_key="celltype"))
        shutil.rmtree('./batch_effect/', ignore_errors=True)
    except:
        print(cells1,cells2)
        pass;
np.array(score).mean()






/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:368: DeprecationWarning:

The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/rpy2/robjects/numpy2ri.py:241: DeprecationWarning:

The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.



Found 106 genes with uniform expression within a single batch (all zeros); these will not be adjusted for batch.


(1425, 452)


/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/scib/metrics/trajectory.py:120: ImplicitModificationWarning:

Trying to modify attribute `.obs` of view, initializing view as actual.

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/scib/metrics/trajectory.py:120: ImplicitModificationWarning:

Trying to modify attribute `.obs` of view, initializing view as actual.

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:368: DeprecationWarning:

The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/rpy2/robjects/numpy2ri.py:241: DeprecationWarning:

The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.



Found 86 genes with uniform expression within a single batch (all zeros); these will not be adjusted for batch.


...

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/scib/metrics/trajectory.py:120: ImplicitModificationWarning:

Trying to modify attribute `.obs` of view, initializing view as actual.

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/scib/metrics/trajectory.py:120: ImplicitModificationWarning:

Trying to modify attribute `.obs` of view, initializing view as actual.

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:368: DeprecationWarning:

The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/rpy2/robjects/numpy2ri.py:241: DeprecationWarning:

The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.



Found 102 genes with uniform expression within a single batch (all zeros); these will not be adjusted for batch.


(1194, 452)


/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/scib/metrics/trajectory.py:120: ImplicitModificationWarning:

Trying to modify attribute `.obs` of view, initializing view as actual.

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/scib/metrics/trajectory.py:120: ImplicitModificationWarning:

Trying to modify attribute `.obs` of view, initializing view as actual.



0.7103639737577235

In [17]:
dataset 2,3
0.7363553783165343 resolution 2
0.6287202582397375 resolution 1
0.8110650815902893 1 10 10
0.925147975679354 1 10 10 10,3


SyntaxError: invalid syntax (2233746869.py, line 1)

In [ ]:
0.8773509847484813

In [ ]:
0.8280230942114288
0.8246860465100816
0.8248684913094871